# USDe Morpho Benchmarks and Reallocation

In this Jupyter notebook, we will automate the process of analyzing and reallocating USDe liquidity within Morpho's financial pools. 

Our primary focus will lie in assessing the current allocation of USDe, the utilization rates, borrowing rates, optimal rates, and capped rates across different liquidity pools distinguished by their Loan-to-Value (LTV) ratios. We aim to ensure that pool allocations are optimized both for the stable functioning of the overall market and for better returns for liquidity providers and borrowers.

1. **Data Analysis**: We will start by performing a data analysis on the current status of various USDe pools. This will include processing important metrics such as total supply, borrow rates, and optimal rates to identify pools that may require attention or reallocation.

2. **Manual Reallocation Criteria**: Based on the analysis, we'll assess the necessity for manual reallocations in accordance with established criteria for active and inactive pools, including factors like minimum balances, maximum utilization, and the safety margin for withdrawing portions of the allocated USDe supply.

In [1]:
import pandas as pd
import numpy as np
import datetime
from data_manipulation import *
from api import *

pd.set_option('display.max_columns', None)

## Manual Reallocation Analysis

Defined Pools we look at for allocation and necessary values

In [2]:
pool_keys = [
    'USDe 94.50%',
    'USDe 91.50%',
    'USDe 86%',
    'USDe 77%',
    'sUSDe 94.50%',
    'sUSDe 91.50%',
    'sUSDe 86%',
    'sUSDe 77%'
]
idle_key = [
    'Idle',
]
idle_col = [
    'LLTV',
    'Total Supply', 
    'Maker Allocation',     
    'Utilization', 
    'Borrow Rate',              
]
columns_ordered = [
    'Status', 
    'LLTV',
    'Total Supply', 
    'Maker Allocation', 
    'Utilization', 
    'Borrow Rate', 
    'Total Borrow', 
    'Maker Borrow', 
    'Optimal Rate', 
    'Capped Borrow Rate', 
    'Inactive Adjustment', 
    'Active Adjustment', 
    'Manual Adjustment',
    'Total Change',
    'Final Allocation',
    'Final Supply',
    'Final Utilization',
    'Final Borrow Rate',
    'Final Capped Rate',
    'Maker Borrow at Old Utilization',
    'Borrow Rate Change',                   
]

pools_df = pd.DataFrame(index=pool_keys, columns=columns_ordered)
idle_df = pd.DataFrame(index=idle_key, columns=idle_col)
pools_df.head(1)

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,Inactive Adjustment,Active Adjustment,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
idle_df

,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate
Idle,NaN,NaN,NaN,NaN,NaN


### Retrieve latest values

Connect to BA API and retrieve the data for the pools we are interested in. First define pools by their uuid, to retrieve the data.

In [4]:
market_to_pool = {
    "db760246f6859780f6c1b272d47a8f64710777121118e56e0cdb4b8b744a3094": 'USDe 94.50%',
    "8e6aeb10c401de3279ac79b4b2ea15fc94b7d9cfc098d6c2a1ff7b2b26d9d02c": 'USDe 91.50%',
    "c581c5f70bd1afa283eed57d1418c6432cbff1d862f94eaf58fdd4e46afbb67f": 'USDe 86%',
    "fd8493f09eb6203615221378d89f53fcd92ff4f7d62cca87eece9a2fff59e86f": 'USDe 77%',
    "e475337d11be1db07f7c5a156e511f05d1844308e66e17d2ba5da0839d3b34d9": 'sUSDe 94.50%',
    "1247f1c237eceae0602eab1470a5061a6dd8f734ba88c7cdc5d6109fb0026b28": 'sUSDe 91.50%',
    "39d11026eae1c6ec02aa4c0910778664089cdd97c3fd23f68f7cd05e2e95af48": 'sUSDe 86%',
    "42dcfb38bb98767afb6e38ccf90d59d0d3f0aa216beb3a234f12850323d17536": 'sUSDe 77%'
}

In [5]:
# market_to_pool.keys()

Retrieve the data from API

In [6]:
# Base URL for market data
base_market_url = "https://morpho-api.blockanalitica.com/markets/{}/historic/?days_ago=0&network=ethereum"
# Base URL for wallet data
base_wallet_url = "https://morpho-api.blockanalitica.com/markets/{}/wallets/?network=ethereum&order=-supply&p_size=4&type=suppliers"
# The wallet address we are interested in
target_wallet = "0x73e65dbd630f90604062f6e02fab9138e713edd9"

api_data = ApiBA(base_market_url, base_wallet_url, market_to_pool.keys(), target_wallet)

With data from API populate the dataframe

In [7]:
market_data = PoolDataHandler(pools_df, market_to_pool)
pools_df = market_data.populate_dataframe(api_data.fetch_data())
pools_df


Updated DataFrame:


,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,Inactive Adjustment,Active Adjustment,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,0.945,1128797,954406,0.93,0.1799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 91.50%,NaN,0.915,54687313,50954861,0.8994,0.1021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 86%,NaN,0.86,98294544,98294544,0.9001,0.0984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 77%,NaN,0.77,7202763,7202763,0.9171,0.1352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 94.50%,NaN,0.945,1971552,1600474,0.8746,0.0939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 91.50%,NaN,0.915,79438972,79438972,0.9013,0.1035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 86%,NaN,0.86,146414875,146414775,0.9047,0.1213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 77%,NaN,0.77,135519,110058,0.2382,0.0211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To see the allocation in the Idle matket:

In [8]:
idle_market_to_pool = {"57f4e42c0707d3ae0ae39c9343dcba78ff79fa663da040eca45717a9b0b0557f": 'Idle'}
idle_api_data = ApiBA(base_market_url, base_wallet_url, idle_market_to_pool.keys(), target_wallet)
idle_data = PoolDataHandler(idle_df, idle_market_to_pool)
idle_df = idle_data.populate_dataframe(idle_api_data.fetch_data())
idle_df


Updated DataFrame:


,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate
Idle,0.0,65112702,65112702,0.0,0.0


### Reallocation Metaparameters

They are set as done by Monet. Change only in case you know what you are doing!

In [9]:
realloc_metaparm = {
    'active_pool' : {
        'min_balance' : 10000000,
        'max_utilization' : 0.9,
        'max_portion_to_withdraw' : 0.1,
    },
    'inactive_pool' : {
        'min_balance' : 100000,
        'max_utilization' : 0.93,
        'max_portion_to_withdraw' : 1,
    },
}

In [10]:
pools_analysis = PoolAnalysis(pools_df, realloc_metaparm)

#### Classify market as Active or Inactive

In the dropdown, select whether the market mentioned should be Active or Inactive by typing in the above cell.  
In a case you defined some of the values wrong, update them with this code:    
`pools_df.loc['pool name', 'column name'] = 'new value'`

In [11]:
# pools_df.loc[pool_key, 'Status'] = 'Active'

In [12]:
pools_analysis.define_active_or_inactive()

Please enter 'Active' or 'Inactive' for each market:
Invalid input. Please enter 'Active' or 'Inactive'.

Updated DataFrame with Status:


### Reallocation Dataframe

Get the final table with all data. For easier overview focus on the following tables

In [13]:
pools_analysis = PoolAnalysis(pools_df, realloc_metaparm)

In [14]:
pools_analysis.update_pool_dataframe()

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,Inactive Adjustment,Active Adjustment,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,Active,0.945,1128797,954406,0.93,0.1799,1049781,887597,0.0947,0.0947,0,8871203,0,8871203,9825609,10000000,0.105,0.032,0.032,9137816,-0.1479
USDe 91.50%,Active,0.915,54687313,50954861,0.8994,0.1021,49185769,45828801,0.1022,0.1021,0,-36458,0,-36458,50918403,54650855,0.9,0.1022,0.1022,45796011,0.0001
USDe 86%,Active,0.86,98294544,98294544,0.9001,0.0984,88474919,88474919,0.0981,0.0981,0,0,0,0,98294544,98294544,0.9001,0.0984,0.0981,88474919,0.0
USDe 77%,Active,0.77,7202763,7202763,0.9171,0.1352,6605653,6605653,0.0894,0.0894,0,2797237,0,2797237,10000000,10000000,0.6606,0.0716,0.0716,9171000,-0.0636
sUSDe 94.50%,Active,0.945,1971552,1600474,0.8746,0.0939,1724319,1399774,0.0959,0.0939,0,8028448,0,8028448,9628922,10000000,0.1724,0.0378,0.0378,8421455,-0.0561
sUSDe 91.50%,Active,0.915,79438972,79438972,0.9013,0.1035,71598345,71598345,0.0996,0.0996,0,0,0,0,79438972,79438972,0.9013,0.1035,0.0996,71598345,0.0
sUSDe 86%,Active,0.86,146414875,146414775,0.9047,0.1213,132461537,132461446,0.1063,0.1063,0,0,0,0,146414775,146414875,0.9047,0.1213,0.1063,132461446,0.0
sUSDe 77%,Active,0.77,135519,110058,0.2382,0.0211,32280,26215,0.047,0.0211,0,9864481,0,9864481,9974539,10000000,0.0032,0.0119,0.0119,2375935,-0.0092


Save the data if you want to perform later analysis or visualizations on this

In [15]:
# pools_analysis.pool_df.to_json(f'reallocations_{datetime.datetime.now().strftime('%Y_%m_%d')}.json')

### Average Utilization, Net and Running Change

Some basic statistics for reallocation

In [16]:
average_utilization = pools_analysis.pool_df['Utilization'].mean()
print(f'The average utilization of the pools is: {average_utilization * 100:.2f}%')

The average utilization of the pools is: 82.07%


The net changes based on the type of adjusment are as follows

In [17]:
net_change_df = pools_analysis.pool_df[['Inactive Adjustment', 'Active Adjustment', 'Manual Adjustment']].agg('sum').to_frame().T
net_change_df.columns = ['Inactive Adjustment Sum', 'Active Adjustment Sum', 'Manual Adjustment Sum']
net_change_df = net_change_df.round(0).astype(int)

net_change_df

,Inactive Adjustment Sum,Active Adjustment Sum,Manual Adjustment Sum
0,0,29524911,0


The running total of changes by the type of adjustment are as follows, which will result in new adjusted Idle supply

In [18]:
cumulative_sums = net_change_df.cumsum(axis=1).iloc[-1]
cumulative_sums_df = pd.DataFrame({
    'Inactive Sum': [cumulative_sums['Inactive Adjustment Sum']],
    'Inactive + Active Sum': [cumulative_sums['Active Adjustment Sum']],
    'Inactive + Active + Manual Sum': [cumulative_sums['Manual Adjustment Sum']]
})
cumulative_sums_df = cumulative_sums_df.round(0).astype(int)

cumulative_sums_df

,Inactive Sum,Inactive + Active Sum,Inactive + Active + Manual Sum
0,0,29524911,29524911


## Manual Reallocation Dataframe

The following table shows the change and new allocated supply with wei allocation

In [19]:
new_pool_data = {
    'Net Change': pools_df['Total Change'],
    'New Allocation Supply': pools_df['Final Allocation']
}
manual_realloaction_pools = pd.DataFrame(new_pool_data)
manual_realloaction_pools['New Allocation Wei'] = manual_realloaction_pools['New Allocation Supply'] * pow(10, 18)

In [20]:
# Create a styled DataFrame
styled_df = manual_realloaction_pools.style.format({
    'Net Change': '{:.0f}',
    'New Allocation Supply': '{:.0f}',
    'New Allocation Wei': '{:.0f}'
}).map(PoolAnalysis.color_net_change, subset=['Net Change'])

styled_df

,Net Change,New Allocation Supply,New Allocation Wei
USDe 94.50%,8871203,9825609,9825609000000000813432832
USDe 91.50%,-36458,50918403,50918402999999998822711296
USDe 86%,0,98294544,98294543999999995667283968
USDe 77%,2797237,10000000,10000000000000000905969664
sUSDe 94.50%,8028448,9628922,9628922000000000408420352
sUSDe 91.50%,0,79438972,79438972000000003683647488
sUSDe 86%,0,146414775,146414775000000001736704000
sUSDe 77%,9864481,9974539,9974538999999999508480000


In [21]:
print(f'The total net change in allocation is: ${manual_realloaction_pools["Net Change"].sum():.0f}')

The total net change in allocation is: $29524911


## Pool Overview Dataframe

A brief statistics for pool overview

In [22]:
pool_stats = [
    'Total Non-Idle Allocation',
    'Supply Weighted LLTV',
    'Supply Weighted sUSDe',
    'Average Borrow Rate',
    'Average Capped Rate',
    'Rate at Prior Equilibrium',
]
columns_stats = [
    'Current',
    'Future',
    'Change'                 
]

pool_overview = pd.DataFrame(index=pool_stats, columns=columns_stats)
statistics = PoolOverview(pools_df, pool_overview)

In [23]:
statistics.update_pool_overview()

,Current,Future,Change
Total Non-Idle Allocation,384970853,414495764,0.076694
Supply Weighted LLTV,0.877484,0.87695,-0.000608
Supply Weighted sUSDe,0.591121,0.592183,0.001796
Average Borrow Rate,0.109559,0.107757,-0.016442
Average Capped Rate,0.101868,0.101155,-0.006994
Rate at Prior Equilibrium,0.109559,0.107757,-0.016442


In [24]:
styled_df = pool_overview.style.format({
    'Current': '{:.3f}',
    'Future': '{:.3f}',
    'Change': '{:.2f}%'
}).map(PoolAnalysis.color_net_change, subset=['Change'])

styled_df

,Current,Future,Change
Total Non-Idle Allocation,384970853.000,414495764.000,0.08%
Supply Weighted LLTV,0.877,0.877,-0.00%
Supply Weighted sUSDe,0.591,0.592,0.00%
Average Borrow Rate,0.110,0.108,-0.02%
Average Capped Rate,0.102,0.101,-0.01%
Rate at Prior Equilibrium,0.110,0.108,-0.02%


### Current Debt Ceiling

In [26]:
debt_ceiling = int(pool_overview.at['Total Non-Idle Allocation', 'Current'] + idle_df['Maker Allocation'].sum())
print(f'The current debt ceiling is: ${debt_ceiling}')

The current debt ceiling is: $450083555
